In [48]:
def fab(max):
    n,a,b = 0,0,1
    while n < max:
        print(b)
        a,b = b, a + b
        n = n + 1

In [49]:
fab(5)

1
1
2
3
5


结果没有问题，但有经验的开发者会指出，直接在 fab 函数中用 print 打印数字会导致该函数可复用性较差，因为 fab 函数返回 None，其他函数无法获得该函数生成的数列。

要提高 fab 函数的可复用性，最好不要直接打印出数列，而是返回一个 List。以下是 fab 函数改写后的第二个版本：

In [50]:
def fab(max):
    n, a, b = 0, 0, 1
    L = []
    while n < max:
        L.append(b)
        a, b = b, a + b
        n = n + 1
    return L

In [51]:
for n in fab(5):
    print(n)

1
1
2
3
5


改写后的 fab 函数通过返回 List 能满足复用性的要求，但是更有经验的开发者会指出，该函数在运行中占用的内存会随着参数 max 的增大而增大，如果要控制内存占用，最好不要用 List

来保存中间结果，而是通过 iterable 对象来迭代。例如，在 Python3.x 中，代码：

In [52]:
 class Fab(object): 

    def __init__(self, max): 
        self.max = max 
        self.n, self.a, self.b = 0, 0, 1 

    def __iter__(self): 
        return self 

    def __next__(self): 
        if self.n < self.max: 
            r = self.b 
            self.a, self.b = self.b, self.a + self.b 
            self.n = self.n + 1 
            return r 
        raise StopIteration()

In [53]:
for n in Fab(5):
    print(n)

1
1
2
3
5


**使用yield**

In [54]:
def fab(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1

**第四个版本的fab和第一版相比，仅仅把print b改为了yield b，就在保持简洁性的同时获得了iterable的效果**

In [55]:
for n in fab(5):
    print(n)

1
1
2
3
5


简单的讲，yield的作用就是把一个函数变成一个generator，带有yield的函数不再是一个普通函数，Python解释器会将其视为一个generator，调用fab(5)不会执行fab函数，而是返回一个generator，调用fab(5)不会执行fab函数，而是返回一个iterable对象！在for循环执行的时，每次循环都会执行fab函数内部的代码，执行到yield b时，fab函数就返回一个迭代值，下次迭代时，代码从yield b的下一条语句继续执行，而函数的本地变量看起来和上次中断执行前是完全一样的，于是函数继续执行，直到再次遇到yield。

In [56]:
f = fab(5)
print(f.__next__())
print(f.__next__())
print(f.__next__())
print(f.__next__())
print(f.__next__())
print(f.__next__())

1
1
2
3
5


StopIteration: 

在函数执行结束时，generator自动抛出StopIteration异常，表示迭代完成。在for循环里，无需处理StopIteration异常，循环会自动结束。

一个带有 yield 的函数就是一个 generator，它和普通函数不同，生成一个 generator 看起来像函数调用，但不会执行任何函数代码，直到对其调用 next()（在 for 循环中会自动调用 next()）才开始执行。虽然执行流程仍按函数的流程执行，但每执行到一个 yield 语句就会中断，并返回一个迭代值，下次执行时从 yield 的下一个语句继续执行。看起来就好像一个函数在正常执行的过程中被 yield 中断了数次，每次中断都会通过 yield 返回当前的迭代值。

yield 的好处是显而易见的，把一个函数改写为一个 generator 就获得了迭代能力，比起用类的实例保存状态来计算下一个 next() 的值，不仅代码简洁，而且执行流程异常清晰。

如何判断一个函数是否是一个特殊的 generator 函数？可以利用 isgeneratorfunction 判断：



In [ ]:
from inspect import isgeneratorfunction
isgeneratorfunction(fab)

要注意区分fab和fab(5)，fab是generator function，而fab(5)是调用fab返回的一个generator，好比类的定义和类的实粒的区别：

In [ ]:
import types
print(isinstance(fab,types.GeneratorType))
print(isinstance(fab(5),types.GeneratorType))

每次调用fab函数都会生成一个新的generator实例，各实例互不影响:

In [60]:
f1 = fab(7)
f2 = fab(5)
print('f1:',f1.__next__())
print('f2:',f2.__next__())
print('f1:',f1.__next__())
print('f2:',f2.__next__())
print('f1:',f1.__next__())
print('f2:',f2.__next__())
print('f1:',f1.__next__())
print('f2:',f2.__next__())
print('f1:',f1.__next__())
print('f2:',f2.__next__())

f1: 1
f2: 1
f1: 1
f2: 1
f1: 2
f2: 2
f1: 3
f2: 3
f1: 5
f2: 5


在一个generator function中，如果没有return，则默认执行至函数执行完毕，如果在执行过程中return，则直接抛出StopIteration终止迭代

In [62]:
def fab(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a + b
        n = n + 1
        return b

In [65]:
f1 = fab(7)
print('f1:',f1.__next__())
print('f1:',f1.__next__())

f1: 1


StopIteration: 1

另一个yield的例子来源于文件读取。如果直接对文件对象调用read()方法,会导致不可预测的内存占用。好的方法是利用固定长度的缓冲区不断读取文件内容。通过yield，我们不再需要编写读文件迭代类，就可以轻松实现文件读取:

In [67]:
def read_file(fpath):
    BLOCK_SIZE = 1024
    with open(fpath,'rb') as f:
        while True:
            block = f.read(BLOCK_SIZE)
            if block:
                yield block
            else:
                return

In [73]:
for a in read_file('YieldExample.ipynb'):
    print(a)

b'{\n "cells": [\n  {\n   "cell_type": "code",\n   "execution_count": 48,\n   "metadata": {\n    "collapsed": false\n   },\n   "outputs": [],\n   "source": [\n    "def fab(max):\\n",\n    "    n,a,b = 0,0,1\\n",\n    "    while n < max:\\n",\n    "        print(b)\\n",\n    "        a,b = b, a + b\\n",\n    "        n = n + 1"\n   ]\n  },\n  {\n   "cell_type": "code",\n   "execution_count": 49,\n   "metadata": {\n    "collapsed": false\n   },\n   "outputs": [\n    {\n     "name": "stdout",\n     "output_type": "stream",\n     "text": [\n      "1\\n",\n      "1\\n",\n      "2\\n",\n      "3\\n",\n      "5\\n"\n     ]\n    }\n   ],\n   "source": [\n    "fab(5)"\n   ]\n  },\n  {\n   "cell_type": "markdown",\n   "metadata": {},\n   "source": [\n    "\xe7\xbb\x93\xe6\x9e\x9c\xe6\xb2\xa1\xe6\x9c\x89\xe9\x97\xae\xe9\xa2\x98\xef\xbc\x8c\xe4\xbd\x86\xe6\x9c\x89\xe7\xbb\x8f\xe9\xaa\x8c\xe7\x9a\x84\xe5\xbc\x80\xe5\x8f\x91\xe8\x80\x85\xe4\xbc\x9a\xe6\x8c\x87\xe5\x87\xba\xef\xbc\x8c\xe7\x9b\xb4\xe6